<center>

**CRIPTOGRAFÍA DE LLAVE PÚBLICA**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

# **🛍️Cifrado de mochila Merkle–Hellman🎒**

<p align="center">
    <img src="https://habrastorage.org/webt/0f/cm/hv/0fcmhvnoq68x7sxrh74dxywgtvo.jpeg"width="400">
</p>

<div align="justify">


**Estructura algebraica**

- **Secuencia super-creciente**  
   Una lista privada de $k$ enteros positivos  
   $$
     \mathbf{a} = (a_1, a_2, \dots, a_k)
   $$
   se dice super-creciente si para todo $j=1,\dots,k$:
   $$
     a_j \;>\; \sum_{i=1}^{j-1} a_i.
   $$
   Este crecimiento garantiza que, dados un entero $b\le\sum_i a_i$ y la secuencia $\mathbf{a}$, existe un único vector binario $(e_1,\dots,e_k)$ tal que
   $$
     b \;=\; \sum_{i=1}^k e_i\,a_i,
     \quad e_i\in\{0,1\},
   $$
   y que puede hallarse con un algoritmo voraz en tiempo $O(k)$.

-  **Enmascarado modular**  
   Para ofuscar $\mathbf{a}$ y crear la clave pública, elegimos:
   - Un módulo $n > \sum_{i=1}^k a_i$.
   - Un entero $u$ tal que $\gcd(u,n)=1$.

   Definimos la secuencia pública
   $$
     a_i^* \;=\; u\;a_i \bmod n,
     \quad i=1,\dots,k.
   $$

- **Cifrado**  
   Dado un bloque de bits $\mathbf{e}=(e_1,\dots,e_k)$, el ciphertext es el entero
   $$
     b \;=\; \sum_{i=1}^k e_i\,a_i^*.
   $$

-  Descifrado
   Quien conoce la secuencia privada $\mathbf{a}$ y $u^{-1}\bmod n$ recupera:

   - $$b' \;=\; b\cdot u^{-1} \bmod n \;=\;\sum_{i=1}^k e_i\,a_i.$$

   - Aplicar el algoritmo voraz: recorrer los $a_j$ de mayor a menor, asignar $$e_j = 1 \quad\text{si}\quad a_j \le b'$$, restar $a_j$ de $b'$ cuando corresponda, y continuar hasta $j=1$.



---
**Modo de uso**

- **Generar clave privada**  
   - Ajusta k (longitud) y Factor super.  
   - Pulsa Generar secuencia → se rellenará el campo Privada aᵢ con una lista super-creciente:
     ```
     a₁, a₂, …, a_k
     ```
   - Por construcción satisface  
     $$
       a_j > \sum_{i=1}^{j-1} a_i
       \quad (j=1,\dots,k).
     $$

- **Configurar parámetros públicos**  
   - Introduce Módulo n, con  
     $$
       n > \sum_{i=1}^k a_i.
     $$  
   - Introduce Multiplicador u, con  
     $$
       1 < u < n
       \quad\text{y}\quad
       \gcd(u,n)=1.
     $$

-  Cifrar un bloque de bits  
   - En el campo Bits (bᵢ) escribe una cadena binaria de longitud k, por ejemplo:
     ```
     01100101
     ```  
   - Pulsa Cifrar → se mostrará:  
     - La clave pública  
       $$
         a_i^* = u\cdot a_i \bmod n
         \quad (i=1,\dots,k),
       $$  
     - El **ciphertext**  
       $$
         b = \sum_{i=1}^k b_i\,a_i^*.
       $$  
   - El valor de b aparecerá también en el control Cifrado b.

- **Descifrar el ciphertext**  
   - Verifica que los campos Privada aᵢ, n y u coincidan con los usados en el cifrado.  
   - En Cifrado b encontrarás el entero b.  
   - Pulsa Descifrar → se mostrará la secuencia de bits recuperada, aplicando:  
     -  
        $$
          b' = b \cdot u^{-1} \bmod n,
        $$  
     - Algoritmo voraz sobre la secuencia super-creciente para hallar
        $$
          b' = \sum_{i=1}^k e_i\,a_i
          \,\Longrightarrow\,
          (e_1,\dots,e_k).
        $$  
   - El resultado debe coincidir con la cadena original en Bits (bᵢ).
</div>

**📥Importaciones📦**

In [ ]:

import random
import ipywidgets as wd
from IPython.display import display, clear_output, HTML

**👨‍💻Implementación👩‍💻**

In [ ]:

def generate_superincreasing(k=8, start=1, factor=2):
    """
    Genera una secuencia super-creciente de longitud k.
    Cada término es > factor * suma de anteriores.
    """
    seq = []
    total = 0
    for _ in range(k):
        term = total * factor + random.randint(1, start)
        seq.append(term)
        total += term
    return seq

def modinv(a, m):
    """Invierte a mod m, lanza ValueError si no existe."""
    g, x, _ = extended_gcd(a, m)
    if g != 1:
        raise ValueError(f"No existe inverso de {a} mod {m}")
    return x % m

def extended_gcd(a, b):
    """Algoritmo extendido de Euclides."""
    if b == 0:
        return (a, 1, 0)
    g, y, x = extended_gcd(b, a % b)
    return (g, x, y - (a // b) * x)

def create_keys(a_priv, u, n):
    """Dada secuencia super-creciente, genera a_pub, inv_u."""
    a_pub = [(u * ai) % n for ai in a_priv]
    inv_u = modinv(u, n)
    return a_pub, inv_u

def encrypt_mh(bits, a_pub):
    """Cifra vector de bits con mochila pública."""
    if len(bits) != len(a_pub):
        raise ValueError("Longitud de bits distinta de la clave pública")
    return sum(b * ap for b, ap in zip(bits, a_pub))

def decrypt_mh(cipher, a_priv, inv_u, n):
    """Descifra: remódulo inv_u y heurística voraz."""
    b = (cipher * inv_u) % n
    bits = [0] * len(a_priv)
    # voraz desde el final
    for i in reversed(range(len(a_priv))):
        if a_priv[i] <= b:
            bits[i] = 1
            b -= a_priv[i]
    return bits

**👨‍💻Implementación👩‍💻**

In [ ]:

style = {'description_width': '120px'}
layout = wd.Layout(width='300px')

# Parámetros de clave privada
k_in   = wd.BoundedIntText(value=8, min=2, max=32, description='k (longitud):', style=style)
factor = wd.BoundedIntText(value=2, min=1, max=10, description='Factor super-: ', style=style)
gen_priv_btn = wd.Button(description='Generar secuencia', button_style='info')

a_priv_in = wd.Textarea(
    value='',
    placeholder='Ej: 2,3,7,14,30,57,120,251',
    description='Privada aᵢ:', layout=wd.Layout(width='600px', height='60px'), style=style
)

n_in = wd.IntText(value=557, description='Módulo n:', style=style)
u_in = wd.IntText(value=31,  description='Multiplicador u:', style=style)

# Parámetros de cifrado
bits_in = wd.Text(description='Bits (bᵢ):', placeholder='Ej: 01100101', style=style)
encrypt_btn = wd.Button(description='Cifrar', button_style='success')

# Parámetros de descifrado
cipher_in  = wd.IntText(description='Cifrado b:', value=0, style=style)
decrypt_btn = wd.Button(description='Descifrar', button_style='warning')

# Salida
out = wd.Output(layout=wd.Layout(border='1px solid #ccc', padding='10px'))

**👨‍💻Implementación👩‍💻**

In [ ]:

def on_gen_priv(_):
    seq = generate_superincreasing(k_in.value, factor=factor.value)
    a_priv_in.value = ','.join(str(x) for x in seq)

def on_encrypt(_):
    with out:
        clear_output()
        try:
            a_priv = [int(x) for x in a_priv_in.value.split(',') if x.strip()]
            n, u = n_in.value, u_in.value
            if u <= 1 or u >= n:
                raise ValueError("u debe estar en 1 < u < n")
            a_pub, inv_u = create_keys(a_priv, u, n)
            bits = [int(b) for b in bits_in.value.strip() if b in '01']
            b = encrypt_mh(bits, a_pub)
            cipher_in.value = b
            display(HTML(f"""
                <h4>Clave pública aᵢ* (mod {n}×{u}):</h4>
                <pre>{a_pub}</pre>
                <b>Cipher b = {b}</b>
            """))
        except Exception as e:
            print("⚠️ Error al cifrar:", e)

def on_decrypt(_):
    with out:
        clear_output()
        try:
            a_priv = [int(x) for x in a_priv_in.value.split(',') if x.strip()]
            n, u, b = n_in.value, u_in.value, cipher_in.value
            a_pub, inv_u = create_keys(a_priv, u, n)
            bits = decrypt_mh(b, a_priv, inv_u, n)
            display(HTML(f"""
                <h4>Descifrado (heurística voraz):</h4>
                <pre>{bits}</pre>
            """))
        except Exception as e:
            print("⚠️ Error al descifrar:", e)

gen_priv_btn.on_click(on_gen_priv)
encrypt_btn.on_click(on_encrypt)
decrypt_btn.on_click(on_decrypt)

**👨‍💻Implementación👩‍💻**

In [ ]:

left = wd.VBox([k_in, factor, gen_priv_btn, a_priv_in, n_in, u_in])
middle = wd.VBox([bits_in, encrypt_btn, cipher_in, decrypt_btn])
ui = wd.HBox([left, middle])
display(HTML("<h2 style='color:#663399'>Cifrado de Mochila (Merkle–Hellman toy)</h2>"))
display(ui, out)


Output(layout=Layout(border='1px solid #ccc', padding='10px'))